# Predicting Pitches
For our project, we'll aim to solve the problem of predicting the next pitch a Major League Baseball Pitcher will throw during an at bat

In [1]:
import utils.utils as utils
import pandas as pd

In [2]:
# get the data
pitch_data = utils.get_pitch_data()
pitch_data.head()

,pitch_data_id,season,pitcher_id,team_id,team_abbrev,era,wins,losses,throws,b1_id,...,is_runner_on_second,is_runner_on_third,runs_pitcher_team,runs_batter_team,game_id,inning_id,half_inning_id,at_bat_id,gid,p1_pitch_type
0,1,2018,605164,110,BAL,None,0,0,R,408045,...,None,None,None,None,8aeda4c6-03a8-4d98-b18c-e2f808c0d0f3,0cef7de8-fc88-44ec-8619-0c3027904cc4,bd439ba2-949a-4e4b-b6ed-54f915ddef43,011a3062-b444-4d3e-96e1-64f19065f2e8,gid_2018_03_29_minmlb_balmlb_1,FF
1,2,2018,592665,142,MIN,None,0,0,R,502143,...,None,None,None,None,8aeda4c6-03a8-4d98-b18c-e2f808c0d0f3,a7d06101-9c1f-4cd6-8851-a97ec42ef555,d787faab-f7e0-466b-872e-e7d535ea2800,040d5908-9ca1-472d-9efd-8a60a250b16e,gid_2018_03_29_minmlb_balmlb_1,FF
2,3,2018,592665,142,MIN,None,0,0,R,502143,...,None,None,None,None,8aeda4c6-03a8-4d98-b18c-e2f808c0d0f3,a7d06101-9c1f-4cd6-8851-a97ec42ef555,d787faab-f7e0-466b-872e-e7d535ea2800,040d5908-9ca1-472d-9efd-8a60a250b16e,gid_2018_03_29_minmlb_balmlb_1,FF
3,4,2018,592665,142,MIN,None,0,0,R,502143,...,None,None,None,None,8aeda4c6-03a8-4d98-b18c-e2f808c0d0f3,a7d06101-9c1f-4cd6-8851-a97ec42ef555,d787faab-f7e0-466b-872e-e7d535ea2800,040d5908-9ca1-472d-9efd-8a60a250b16e,gid_2018_03_29_minmlb_balmlb_1,FF
4,5,2018,592665,142,MIN,None,0,0,R,502143,...,None,None,None,None,8aeda4c6-03a8-4d98-b18c-e2f808c0d0f3,a7d06101-9c1f-4cd6-8851-a97ec42ef555,d787faab-f7e0-466b-872e-e7d535ea2800,040d5908-9ca1-472d-9efd-8a60a250b16e,gid_2018_03_29_minmlb_balmlb_1,SL


In [3]:
pitch_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 72 columns):
pitch_data_id          326 non-null int64
season                 326 non-null int64
pitcher_id             326 non-null int64
team_id                326 non-null object
team_abbrev            326 non-null object
era                    0 non-null object
wins                   326 non-null int64
losses                 326 non-null int64
throws                 326 non-null object
b1_id                  326 non-null int64
b1_team_id             326 non-null object
b1_stand               326 non-null object
b1_height              326 non-null int64
b1_bats                326 non-null object
b1_avg                 326 non-null float64
b1_hr                  326 non-null int64
b1_rbi                 326 non-null int64
b1_bat_order           304 non-null float64
b1_game_position       304 non-null object
p1_pitch_id            326 non-null object
p0_pitch_id            316 non-null object
p

In [ ]:
cols_to_drop=['pitch_data_id','season','team_id','game_id','inning_id','half_inning_id','at_bat_id','gid']
pitch_data = utils.drop_columns_by_list(pitch_data,cols_to_drop)